In [1]:
from src.guitar.Guitar import Guitar
from src.hand.LeftFinger import LeftFinger
from src.hand.LeftHand import LeftHand
from src.guitar.GuitarString import createGuitarStrings
from src.midi.midiToNote import midiToGuitarNotes,processedNotes
from src.utils.utils import convertNotesToChord, convertChordTofingerPositions
from src.HandPoseRecorder import HandPoseRecorder,HandPoseRecordPool
import copy
import time

## Import Midi file 导入Midi文件
Import midi file and convert it to a list of dict which including notes and beat. 导入midi文件，并将其转换为一个列表，这个列表里的元素是包含音符和节拍信息的字典。

In [2]:
midiFilePath = "asset/Corridors Of Time Fingerstyle.mid"
filename = midiFilePath.split("/")[-1].split(".")[0]

GuitarNotes = midiToGuitarNotes(midiFilePath,useChannel=1)

GuitarNotes

MidiTrack([
  Message('note_on', channel=0, note=50, velocity=76, time=0),
  MetaMessage('track_name', name='Steel Guitar', time=0),
  Message('program_change', channel=0, program=25, time=0),
  MetaMessage('key_signature', key='C', time=0),
  Message('control_change', channel=0, control=101, value=0, time=0),
  Message('control_change', channel=0, control=100, value=0, time=0),
  Message('control_change', channel=0, control=6, value=6, time=0),
  Message('note_off', channel=0, note=50, velocity=64, time=240),
  Message('note_on', channel=0, note=54, velocity=76, time=0),
  Message('note_off', channel=0, note=54, velocity=64, time=240),
  Message('note_on', channel=0, note=59, velocity=76, time=0),
  Message('note_off', channel=0, note=59, velocity=64, time=240),
  Message('note_on', channel=0, note=61, velocity=76, time=0),
  Message('note_off', channel=0, note=61, velocity=64, time=240),
  Message('note_on', channel=0, note=66, velocity=76, time=0),
  Message('note_off', channel=0, n

[{'notes': [50], 'real_time': 0.0},
 {'notes': [54], 'real_time': 0.3},
 {'notes': [59], 'real_time': 0.6},
 {'notes': [61], 'real_time': 0.8999999999999999},
 {'notes': [66], 'real_time': 1.2},
 {'notes': [54], 'real_time': 1.5},
 {'notes': [59], 'real_time': 1.8},
 {'notes': [61], 'real_time': 2.1},
 {'notes': [64], 'real_time': 2.4},
 {'notes': [54], 'real_time': 2.6999999999999997},
 {'notes': [59], 'real_time': 2.9999999999999996},
 {'notes': [61], 'real_time': 3.2999999999999994},
 {'notes': [66], 'real_time': 3.599999999999999},
 {'notes': [54], 'real_time': 3.899999999999999},
 {'notes': [59], 'real_time': 4.199999999999999},
 {'notes': [61], 'real_time': 4.499999999999999},
 {'notes': [50], 'real_time': 4.799999999999999},
 {'notes': [54], 'real_time': 5.099999999999999},
 {'notes': [59], 'real_time': 5.399999999999999},
 {'notes': [61], 'real_time': 5.699999999999998},
 {'notes': [66], 'real_time': 5.999999999999998},
 {'notes': [54], 'real_time': 6.299999999999998},
 {'notes

## init guitar and handPoseRecordPool 初始化吉他和handPoseRecordPool
init guitar with a list of string. here we use a standard 6-string guitar. 用一个字符串列表来初始化吉他。这里我们使用标准的6弦吉他。
init the start hand pose. here we put a hand at barrel 0 and put fingers on string 3, which is G string. 初始化起始手型。在这里，我们将设定初始手型位置为第0品，并将手指放在第3弦上，也就是G弦上。

handPoseRecordPool can init with param `size`, which is the size of the pool. Bigger size will cost more memory and more time but can calculate more possibilities. handPoseRecordPool可以通过参数`size`初始化，即池的大小。越大就会消耗更多的内存和时间，但可以计算更多可能性。

In [3]:
# 设定各弦音高
guitar_string_list = createGuitarStrings(["e", "b", "G", "D", "A", "E1"])
# 初始化吉它
guitar = Guitar(guitar_string_list)
# 设定各手指状态
leftFingers = [
    LeftFinger(1, guitar_string_list[1], 1),
    LeftFinger(2, guitar_string_list[2], 2),
    LeftFinger(3, guitar_string_list[3], 3),
    LeftFinger(4, guitar_string_list[4], 4)
]
# 初始化左手
initLeftHand = LeftHand(leftFingers)
# 初始化第一个记录器
handPoseRecord = HandPoseRecorder()
handPoseRecord.addHandPose(initLeftHand, 0,0)
# 初始化记录池
handPoseRecordPool = HandPoseRecordPool(20)
handPoseRecordPool.append(handPoseRecord)

## Calculate the possibility of hand pose 计算手型的可能性
- For each notes, we calculate all possibile chords, which includes position infomations of notes on guitar. 对于每个音符，我们计算所有可能的和弦，包括音符在吉他上的位置信息。
- For each chords, we calculate all possibile hand poses, which includes the position of fingers on guitar.
- For each hand poses, we generate a LeftHand object if it is possible to play. 对于每个和弦，我们计算所有可能的手型，包括手指在吉他上的位置。
- For each LeftHand object, we calculate entropy, the cost of switch hand pose. Then we append it to the handPoseRecord. 对于每个手型，如果可以演奏，我们就生成一个LeftHand对象。
- For each handPoseRecord, we put it in the handPoseRecordPool if its entropy is lower than the record in the pool. 对于每个LeftHand对象，我们计算熵，即切换手型的成本。然后将其添加到handPoseRecord中。
- After all notes are calculated, we get the best handPoseRecord from the pool. Then print the result. 计算完所有音符后，我们从池中获取最佳handPoseRecord，然后打印结果


In [4]:
for i in range(len(GuitarNotes)):
    start_time = time.time()
    guitarNote = GuitarNotes[i]
    notes = guitarNote["notes"]
    real_time = guitarNote["real_time"]
    notes = processedNotes(notes)
    
    # calculate all possible chords, including the position information of notes on the guitar. 计算所有可能的和弦,包含音符在吉它上的位置信息。
    chords = convertNotesToChord(notes, guitar)
    
    # init current record list. 记录池先更新初始化当前记录列表。
    handPoseRecordPool.readyForRecord()
        
    # calculate all possible fingerings, including the position information of fingers on the guitar. 计算所有可能的按法，包含手指在吉它上的位置信息。
    for chord in chords:
        fingerPositionsList = convertChordTofingerPositions(chord)
        for fingerPositions in fingerPositionsList:
            # Iterate through all recorders in the record pool. 遍历记录池中的所有记录器。
            for handPoseRecord in handPoseRecordPool.preHandPoseRecordPool:
                oldhand = handPoseRecord.currentHandPose()
                # Iterate through the list of fingerings, generate a new LeftHand object based on the fingering. 遍历按法列表，根据按法生成新的LeftHand对象。
                hand, entropy = oldhand.generateNextHands(
                    guitar, fingerPositions)
                if hand is None:
                    continue
                newHandPoseRecord = copy.deepcopy(handPoseRecord)
                newHandPoseRecord.addHandPose(hand, entropy, real_time)
                handPoseRecordPool.append(newHandPoseRecord)
    
    end_time = time.time()
    print(f"当前时间是{real_time}，当前有{len(handPoseRecordPool.curHandPoseRecordPool)}个rerocder，耗时{end_time-start_time:.2f}秒。")


# after all iterations, read the best solution in the record pool. 全部遍历完以后，读取记录池中的最优解。
bestHandPoseRecord = handPoseRecordPool.curHandPoseRecordPool[0]
bestEntropy = bestHandPoseRecord.currentEntropy
print(f"最小消耗熵为：{bestEntropy}")
bestHandPoseRecord.output(True)
bestHandPoseRecord.save(f"output/{filename}.json")

当前时间是0.0，当前有1个rerocder，耗时0.00秒。
当前时间是0.3，当前有4个rerocder，耗时0.00秒。
当前时间是0.6，当前有14个rerocder，耗时0.01秒。
当前时间是0.8999999999999999，当前有9个rerocder，耗时0.03秒。
当前时间是1.2，当前有17个rerocder，耗时0.02秒。
当前时间是1.5，当前有20个rerocder，耗时0.04秒。
当前时间是1.8，当前有2个rerocder，耗时0.06秒。
当前时间是2.1，当前有2个rerocder，耗时0.01秒。
当前时间是2.4，当前有2个rerocder，耗时0.01秒。
当前时间是2.6999999999999997，当前有7个rerocder，耗时0.01秒。
当前时间是2.9999999999999996，当前有5个rerocder，耗时0.03秒。
当前时间是3.2999999999999994，当前有11个rerocder，耗时0.03秒。
当前时间是3.599999999999999，当前有5个rerocder，耗时0.06秒。
当前时间是3.899999999999999，当前有20个rerocder，耗时0.02秒。
当前时间是4.199999999999999，当前有9个rerocder，耗时0.12秒。
当前时间是4.499999999999999，当前有16个rerocder，耗时0.08秒。
当前时间是4.799999999999999，当前有3个rerocder，耗时0.06秒。
当前时间是5.099999999999999，当前有12个rerocder，耗时0.01秒。
当前时间是5.399999999999999，当前有20个rerocder，耗时0.10秒。
当前时间是5.699999999999998，当前有20个rerocder，耗时0.16秒。
当前时间是5.999999999999998，当前有20个rerocder，耗时0.17秒。
当前时间是6.299999999999998，当前有20个rerocder，耗时0.16秒。
当前时间是6.599999999999998，当前有1个rerocder，耗时0.18秒。
当前时间是6.899999999999998，当前有2个rerocder，耗时


## Initialize Animation Values 初始化动画所需数值

In Blender, by determining the positions of the following key points: IK pivot point, Y-axis rotation value, we can determine the entire left hand's posture. These key points are:
在blender中，通过确定以下几个关键点的位置，ik轴点，y轴旋转值，就可以确定整个左手的状态，这几个关键点分别是：
- Palm 手掌
- Thumb 大拇指
- Index finger 食指
- Middle finger 中指
- Ring finger 无名指
- Pinky finger 小指

There are four extreme positions of the hand shape, which are:
而手型有四个极端位置，分别是：
- Open string position on the 1st string 1弦空品
- Open string position on the 6th string 6弦空品
- 12th fret position on the 1st string 1弦12品
- 12th fret position on the 6th string 6弦12品

For each extreme position, we can adopt three methods to press the strings:
而每个极端位置，我们都可以采用三种方式来按弦，分别是：
- Index finger and middle finger on different frets, with the whole palm parallel to the neck 食指和中指在不同品上，整个手掌平行于琴颈
- Index finger and middle finger on the same fret, with the index finger pointing outward and the middle finger pointing inward, requiring the palm to rotate at a certain angle 食指与中指同品，而食指向外，中指向内，此时手掌要旋转一定角度
- Index finger and middle finger on the same fret, with the index finger pointing inward and the middle finger pointing outward, requiring the palm to rotate at a certain angle 食指与中指同品，而食指向内，中指向外，此时手掌要旋转一定角度

In Blender, we record the values of all key points for these four extreme positions in three different ways, totaling 12 hand shapes. 在blender中，我们记录下来以上四个极端位置的三种不同方式下，所有关键点的值，共计12种手型。

Through these 12 hand shapes, we can obtain the hand shape at any position through interpolation. 通过这12种手型，我们可以通过插值的方式，得到任意位置的手型。

In [5]:

# frames per second. blender文件中的动画帧率
FPS = 24

midiFilePath = "asset/Corridors Of Time Fingerstyle.mid"
filename = midiFilePath.split("/")[-1].split(".")[0]

recorder = f"output/{filename}.json"
animation = f"output/{filename}_animation.json"

In [6]:
from src.animate.animate import hand2Animation
hand2Animation(recorder, animation, FPS)